In [1]:
!pip install tensorflow numpy pillow


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image as kp_image
from tensorflow.keras.applications.vgg19 import preprocess_input
import numpy as np
from PIL import Image

In [3]:

# Load and preprocess images
def load_and_process_img(path_to_img):
    img = kp_image.load_img(path_to_img, target_size=(224, 224))
    img = kp_image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return tf.convert_to_tensor(img)

In [4]:


def deprocess_img(processed_img):
    img = processed_img.numpy().squeeze()
    img = img[:, :, ::-1]  # Convert from BGR to RGB
    img = img + 103.939
    img = img[:, :, ::-1]  # Convert back to RGB
    img = np.clip(img, 0, 255).astype('uint8')
    return img

In [5]:

# Define the model
def get_model():
    model = VGG19(weights='imagenet', include_top=False)
    model.trainable = False
    return model

In [6]:

# Define loss functions
def get_loss(model):
    # Define the layers for content and style features
    content_layers = ['block5_conv2']
    style_layers = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block5_conv1']
    
    # Extract features from the model
    content_outputs = [model.get_layer(name).output for name in content_layers]
    style_outputs = [model.get_layer(name).output for name in style_layers]
    
    model_outputs = content_outputs + style_outputs
    model = tf.keras.Model(model.input, model_outputs)
    
    def content_loss(base_content, target):
        return tf.reduce_mean(tf.square(base_content - target))
    
    def style_loss(base_style, target):
        base_style = tf.reshape(base_style, [-1])
        target = tf.reshape(target, [-1])
        return tf.reduce_mean(tf.square(base_style - target))
    
    return content_layers, style_layers, content_loss, style_loss, model

In [7]:



# Load images
style_path = 'test_image1.jpg'
content_path = 'test_image2.jpg'
content_img = load_and_process_img(content_path)
style_img = load_and_process_img(style_path)

# Create the model
model = get_model()
content_layers, style_layers, content_loss, style_loss, model = get_loss(model)

# Initialize the generated image
generated_img = tf.Variable(content_img, dtype=tf.float32)

# Define optimizers and training loop
optimizer = tf.optimizers.Adam(learning_rate=0.02)

In [8]:
@tf.function
def train_step():
    with tf.GradientTape() as tape:
        generated_features = model(generated_img)
        content_loss_value = content_loss(generated_features[0], model(content_img)[0])
        style_loss_value = 0
        for i, layer_name in enumerate(style_layers):
            style_output = generated_features[len(content_layers) + i]
            target_style = model(style_img)[len(content_layers) + i]
            style_loss_value += style_loss(style_output, target_style)
        style_loss_value *= 0.1
        total_loss = content_loss_value + style_loss_value
    grads = tape.gradient(total_loss, generated_img)
    optimizer.apply_gradients([(grads, generated_img)])
    return total_loss

In [9]:

# Train the model
epochs = 1000
for epoch in range(epochs):
    loss = train_step()
    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.numpy()}')

# Convert generated image to PIL format and display
# generated_img = deprocess_img(generated_img)
# Image.fromarray(generated_img).show()

# Convert generated image to PIL format and save locally
generated_img = deprocess_img(generated_img)
generated_img_pil = Image.fromarray(generated_img)
generated_img_pil.save('generated_image.png')